Absolute magnitude is defined to be the apparent magnitude an object would have if it were located at a distance of 10 parsecs.
In astronomy, absolute magnitude (M) is a measure of the luminosity of a celestial object on an inverse logarithmic astronomical magnitude scale.

In [1]:
import time
import numpy as np
import h5py
import hdf5plugin
import pandas as pd
import os 
import csv
import resource

import matplotlib as mpl
import matplotlib.colors as cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from unyt import yr, Myr, kpc, arcsec, nJy, Mpc, Msun, erg, s, Hz
from astropy.cosmology import Planck18 as cosmo


from synthesizer.grid import Grid
from synthesizer.particle import Gas, Stars
from synthesizer.filters import Filter, FilterCollection
from synthesizer.sed import combine_list_of_seds
from synthesizer.load_data.load_camels import load_CAMELS_IllustrisTNG
from synthesizer.kernel_functions import Kernel
from synthesizer.conversions import lnu_to_absolute_mag

current_directory = os.getcwd()
print("Current directory:", current_directory)


Current directory: /home/jovyan/camels/LH/get_LF


# Start the timer
start_time = time.time()


In [2]:

# Alternative method for LF:
# try this method again, but using AB mag instead of mass, and suply your own bins (up to -17, say)
def calc_lf(ab_mag, volume, massBinLimits):
# OG:        hist, dummy = np.histogram(np.log10(mstar), bins = massBinLimits)
        hist, dummy = np.histogram(ab_mag, bins = massBinLimits)
        hist = np.float64(hist)
        phi = (hist / volume) / (massBinLimits[1] - massBinLimits[0])
        phi_sigma = (np.sqrt(hist) / volume) /\
                    (massBinLimits[1] - massBinLimits[0]) # Poisson errors
        return phi, phi_sigma, hist
    


phi_arr =[] #phi
phi_sigma_arr =[] # phi_sigma
hist_arr = [] # hist
z_arr = [] #redshift_074, 
abs_mag_arr = [] #absolute mag (th filter)
Vcom_arr = [] # comoving vol
massBinLimits_arr = []


len(phi_arr)

In [3]:
grid_name = "bc03-2016-Miles_chabrier-0.1,100.hdf5"
grid_dir = "/home/jovyan/"

# Create a new grid
#grid = Grid(grid_name, grid_dir=grid_dir, read_lines=False)

# instead of using a filter, which requires us to load in large SEDs first, pass the grid wavelength
#filt1 = Filter("top_hat/filter.1", lam_min=1400, lam_max=1600, new_lam=grid.lam)
#filt_lst = [filt1]

# Define a new set of wavelengths
lims_lams=(1400, 1600)

grid = Grid(grid_name, grid_dir=grid_dir, lam_lims=lims_lams, read_lines=False)
print(grid.shape)
print(grid)

(221, 7, 401)
------------------------------
SUMMARY OF GRID
log10age: [ 5.          5.          5.14998846  5.20000187  5.25000503  5.3000082
  5.3499959   5.40000235  5.45000263  5.50000307  5.55000809  5.60000309
  5.65000623  5.7000024   5.74999898  5.80000183  5.85000259  5.90000097
  5.95000442  6.          6.01998816  6.04000864  6.06001754  6.08001285
  6.09999123  6.12001425  6.14000495  6.15998805  6.18001112  6.20000187
  6.22000343  6.23999979  6.2599998   6.28000895  6.3000082   6.3200008
  6.34000762  6.36000613  6.37999404  6.40000235  6.42000528  6.43999547
  6.45999526  6.48000694  6.50000307  6.51999853  6.54000394  6.56000233
  6.58000069  6.60000309  6.62000064  6.64000415  6.66000172  6.67999992
  6.7000024   6.7200021   6.73999931  6.76000005  6.7800003   6.80000183
  6.81999773  6.83999939  6.86000242  6.88000139  6.90000097  6.91999803
  6.93999821  6.9599996   6.97999882  7.          7.01998816  7.04000864
  7.06001754  7.08001285  7.09999123  7.11998131  7.140

In [4]:
# Define the directory where the text files will be saved
output_dir = "/home/jovyan/camels/LH/get_LF/output/"


In [5]:

# get gals
LH_X = 'LH_104'
dir_ = '/home/jovyan/Data/Sims/IllustrisTNG/LH/' + LH_X
gals_074 = load_CAMELS_IllustrisTNG(
    dir_,
    snap_name='snapshot_074.hdf5', 
    fof_name='groups_074.hdf5',
    verbose=False,
)

/srv/conda/envs/notebook/lib/python3.9/site-packages/synthesizer/particle/galaxy.py:110: RuntimeWarning: In `load_stars`: one of either `initial_masses`, `ages` or `metallicities` is not provided, setting `stars` object to `None`
  self.load_stars(stars=stars)
/srv/conda/envs/notebook/lib/python3.9/site-packages/synthesizer/particle/galaxy.py:111: RuntimeWarning: In `load_gas`: one of either `masses` or `metallicities` is not provided, setting `gas` object to `None`
  self.load_gas(gas=gas)
/srv/conda/envs/notebook/lib/python3.9/site-packages/unyt/array.py:1938: RuntimeWarning: invalid value encountered in divide
  out_arr = func(
/srv/conda/envs/notebook/lib/python3.9/site-packages/unyt/array.py:1813: RuntimeWarning: invalid value encountered in log10
  out_arr = func(np.asarray(inp), out=out_func, **kwargs)


LH_104 = 'LH_104'
dir_ = '/home/jovyan/Data/Sims/IllustrisTNG/LH/' + LH_104
gals_104 = load_CAMELS_IllustrisTNG(
    dir_,
    snap_name='snapshot_074.hdf5', 
    fof_name='groups_074.hdf5',
    verbose=False,
)


for gal in gals_074:
    print(f"Galaxy has {gal.stars.nstars} stellar particles")
    print(f"Galaxy gas {gal.gas.nparticles} gas particles")


gals_074

In [6]:

# Filter galaxies to only include those with 100 or more star particles
gals_074 = [gal for gal in gals_074 if gal.stars.nstars >= 100]
print(len(gals_074))
gals_074 = [gal for gal in gals_074 if gal.gas.nparticles >= 100]


147


In [7]:
# -1.68397285e+02
for gal in gals_074:
    print(gal.stars.ages)

[5.19399790e+09 7.28023390e+09 6.15129966e+09 ... 5.79615769e+09
 6.17312294e+09 6.06045536e+09] yr
[5.30218990e+09 2.86554961e+09 4.72192391e+09 ... 6.62129701e+09
 6.37679433e+09 5.48050889e+09] yr
[7.49560537e+09 7.46622549e+09 4.98582701e+09 ... 4.70777203e+09
 5.85295072e+09 4.72192391e+09] yr
[5.09408414e+09 6.27689596e+09 6.34171036e+09 ... 7.76165199e+09
 5.52844574e+09 4.73779116e+09] yr
[7.84442403e+09 4.79896746e+09 6.32843099e+09 ... 3.35317717e+09
 4.21918543e+09 5.15194032e+09] yr
[7.21781542e+09 7.37692196e+09 6.47504390e+09 ... 5.32971848e+09
 2.03989136e+09 3.79933056e+09] yr
[7.63792810e+09 3.82889640e+09 7.72185364e+09 1.23547067e+09
 7.37165186e+09 6.25873440e+09 1.49283604e+09 2.16292926e+09
 4.14018159e+09 1.92057601e+09 5.20424540e+09 4.17291589e+09
 1.43258814e+09 5.68363236e+09 8.33567770e+09 7.35038602e+09
 6.66228519e+09 1.12492265e+09 7.27075831e+09 7.33424052e+09
 1.10688252e+09 1.15491954e+09 6.28818496e+09 7.16313872e+09
 7.17212527e+09 6.22895866e+09 5.2

[2.61821111e+09 5.81343491e+09 2.28813755e+09 5.39916368e+09
 2.13873493e+09 4.84730575e+09 6.13526693e+09 3.57926520e+09
 2.57118243e+09 2.55331132e+09 4.08416364e+09 5.59985522e+09
 6.35492306e+09 5.00075725e+09 6.42853440e+09 8.28035231e+09
 2.19775334e+09 5.39017929e+09 3.12823104e+09 4.08818488e+09
 5.83983343e+09 4.43410041e+09 7.56180107e+09 2.04397582e+09
 4.55648202e+09 4.35458471e+09 5.43784504e+09 6.71012837e+09
 5.31750860e+09 7.42607915e+09 5.49655964e+09 6.20815272e+09
 5.62239856e+09 6.92256573e+09 6.84941588e+09 2.67277589e+08
 5.88421092e+09 5.20896619e+09 8.69483312e+08 2.17365946e+09
 4.95245505e+09 4.97251002e+09 6.73273107e+09 6.12572302e+09
 3.51740976e+09 6.90193552e+09 3.44145359e+09 5.00902936e+09
 8.28035231e+09 3.89590542e+09 6.23815576e+09 6.07508765e+09
 2.52386061e+09 7.73993647e+09 6.56144924e+09 5.89456148e+09
 6.94974152e+09 3.52184889e+09 6.81724545e+09 6.84586583e+09
 6.91571250e+09 6.74021366e+09 6.08237606e+09 4.63203695e+07
 6.82981558e+09 6.487461

# commented out to see if Chris fix and my environment are working:

#gals_cp = [gal for gal in gals_cp if gal.stars.ages >= 0] # found some negative data in gal[276] for LH_50, try this to fix.



# Assuming gals_cp is a list or array containing multiple galaxies
for gal in gals_074:
    # Access the Stars object for the current galaxy
    stars = gal.stars
    
    # Access the relevant attributes
    initial_masses = stars.initial_masses
    ages = stars.ages
    metallicities = stars.metallicities

    # Create a mask for stars with non-negative initial_masses and positive ages
    mask = (initial_masses >= 0) & (ages > 0)
    
    # Apply the mask to all relevant attributes
    cleaned_initial_masses = initial_masses[mask]
    cleaned_ages = ages[mask]
    cleaned_metallicities = metallicities[mask]
    
    # Create a new Stars object with the cleaned data
    cleaned_stars = Stars(initial_masses=cleaned_initial_masses, ages=cleaned_ages, metallicities=cleaned_metallicities)
    
    # Update the stars attribute of the current galaxy with the cleaned Stars object
    gal.stars = cleaned_stars

    # Optionally print the cleaned ages to verify
    print(cleaned_stars.ages)



#gals_074 = [gal for gal in gals_074 if gal.stars.ages >= 100] # found some negative data in gal[276] for LH_50, try this to fix.
gals_test = [gal for gal in gals_074 if all(age >= 100 for age in gal.stars.ages)]


In [8]:
for gal in gals_074:
    print(gal.stars.ages)

[5.19399790e+09 7.28023390e+09 6.15129966e+09 ... 5.79615769e+09
 6.17312294e+09 6.06045536e+09] yr
[5.30218990e+09 2.86554961e+09 4.72192391e+09 ... 6.62129701e+09
 6.37679433e+09 5.48050889e+09] yr
[7.49560537e+09 7.46622549e+09 4.98582701e+09 ... 4.70777203e+09
 5.85295072e+09 4.72192391e+09] yr
[5.09408414e+09 6.27689596e+09 6.34171036e+09 ... 7.76165199e+09
 5.52844574e+09 4.73779116e+09] yr
[7.84442403e+09 4.79896746e+09 6.32843099e+09 ... 3.35317717e+09
 4.21918543e+09 5.15194032e+09] yr
[7.21781542e+09 7.37692196e+09 6.47504390e+09 ... 5.32971848e+09
 2.03989136e+09 3.79933056e+09] yr
[7.63792810e+09 3.82889640e+09 7.72185364e+09 1.23547067e+09
 7.37165186e+09 6.25873440e+09 1.49283604e+09 2.16292926e+09
 4.14018159e+09 1.92057601e+09 5.20424540e+09 4.17291589e+09
 1.43258814e+09 5.68363236e+09 8.33567770e+09 7.35038602e+09
 6.66228519e+09 1.12492265e+09 7.27075831e+09 7.33424052e+09
 1.10688252e+09 1.15491954e+09 6.28818496e+09 7.16313872e+09
 7.17212527e+09 6.22895866e+09 5.2

[3.59463084e+09 3.34918647e+09 5.11507331e+09 3.61651440e+09
 6.69683581e+09 7.33017767e+09 3.43920564e+09 5.72475033e+09
 5.45408731e+09 3.61432994e+09 7.67009418e+09 5.86340542e+09
 5.48489290e+09 6.09206546e+09 6.13049899e+09 6.80006438e+09
 3.87713969e+09 3.82257412e+09 6.36588188e+09 7.12825249e+09
 4.37173158e+09 6.60548741e+09 7.14957056e+09 4.27183955e+09
 5.17580136e+09 6.12333174e+09 3.61432994e+09 3.57266805e+09
 6.73085635e+09 5.77878733e+09 6.03590483e+09 6.99310769e+09
 6.12572302e+09 4.47896431e+09 4.12224050e+09 7.27075831e+09
 6.93535089e+09 3.36797528e+09 3.57266805e+09 6.77724259e+09
 3.71615729e+09 3.60995831e+09 7.21344763e+09 3.31201632e+09
 3.93324880e+09 6.81183512e+09 6.05066020e+09 6.69111422e+09
 7.54555581e+09 3.85411784e+09 6.08964599e+09 6.67962723e+09
 5.29296863e+09 4.20348032e+09 7.04515867e+09 6.55335306e+09
 6.16606380e+09 4.09822378e+09 6.07994851e+09 6.65259280e+09
 8.22608992e+09 6.86707364e+09 7.88866975e+09 6.38982827e+09
 7.71168304e+09 7.184034

[2.61821111e+09 5.81343491e+09 2.28813755e+09 5.39916368e+09
 2.13873493e+09 4.84730575e+09 6.13526693e+09 3.57926520e+09
 2.57118243e+09 2.55331132e+09 4.08416364e+09 5.59985522e+09
 6.35492306e+09 5.00075725e+09 6.42853440e+09 8.28035231e+09
 2.19775334e+09 5.39017929e+09 3.12823104e+09 4.08818488e+09
 5.83983343e+09 4.43410041e+09 7.56180107e+09 2.04397582e+09
 4.55648202e+09 4.35458471e+09 5.43784504e+09 6.71012837e+09
 5.31750860e+09 7.42607915e+09 5.49655964e+09 6.20815272e+09
 5.62239856e+09 6.92256573e+09 6.84941588e+09 2.67277589e+08
 5.88421092e+09 5.20896619e+09 8.69483312e+08 2.17365946e+09
 4.95245505e+09 4.97251002e+09 6.73273107e+09 6.12572302e+09
 3.51740976e+09 6.90193552e+09 3.44145359e+09 5.00902936e+09
 8.28035231e+09 3.89590542e+09 6.23815576e+09 6.07508765e+09
 2.52386061e+09 7.73993647e+09 6.56144924e+09 5.89456148e+09
 6.94974152e+09 3.52184889e+09 6.81724545e+09 6.84586583e+09
 6.91571250e+09 6.74021366e+09 6.08237606e+09 4.63203695e+07
 6.82981558e+09 6.487461

In [9]:
# Iterate through each galaxy in gals_074 with an index
for index, gal in enumerate(gals_074):
    # Access the ages array of the stars in the current galaxy
    ages = gal.stars.ages
    
    # Find the negative values in the ages array
    negative_ages = ages[ages < 0]
    
    # If there are any negative values, print them
    if negative_ages.size > 0:
        print(f"Galaxy Index: {index}")
        print("Negative ages:", negative_ages)


Galaxy Index: 18
Negative ages: [-168.39728545] yr
Galaxy Index: 39
Negative ages: [-168.39728545] yr
Galaxy Index: 69
Negative ages: [-168.39728545] yr
Galaxy Index: 94
Negative ages: [-168.39728545] yr


gals_cp = gals_074 

# Remove stars from each galaxy where the age is less than 100
for gal in gals_cp:
    obj = gal.stars.ages
    #print(obj)
    cleaned_obj = [arr[arr >= 0] for arr in obj]
    gal.stars.ages = cleaned_obj
    print(gal.stars.ages)
    


gals_test = [gal for gal in gals_074 if all(metallicities >=1e-6 for metallicities in gal.stars.metallicities)]

# found some very small metalcities also (7.16442446e-06 to -10)
#gals_074 = [gal for gal in gals_074 if gal.stars.metallicities >=1e-6]

In [10]:
for gal in gals_074:
    print(gal.gas.nparticles)
    print(gal.stars.nstars)

359109
8530
338
1482
360439
12449
141603
4459
3682
1065
331
1744
2618
254
168
161
117
170
86174
3040
1469
853
541
313
21846
2217
17215
1424
257
163
58526
1140
2690
384
210
405
58940
1840
39707
1499
5282
492
44859
693
681
514
1039
175
30393
1772
31171
1874
401
193
32842
923
2006
369
23079
594
6023
512
24812
1006
732
140
25714
2158
21714
2043
30245
1632
20632
660
3377
289
21009
1709
26850
2129
21572
1329
22860
1187
24267
617
152
294
22625
1264
18701
1693
21879
1179
22162
486
698
402
22470
1550
22168
662
182
100
17042
1019
18636
1029
17732
1319
415
260
16478
247
143
113
15953
546
10431
427
15169
577
12558
1256
12766
449
12483
354
7423
606
11319
647
12972
1052
2256
283
1799
234
11002
194
10661
783
10420
449
10685
508
10088
808
10426
378
10005
559
10519
337
8438
393
7224
841
9336
249
8653
252
6447
450
4447
262
528
123
7712
322
7884
286
5746
191
7637
293
6281
244
7545
345
6113
278
743
137
5533
354
6508
128
5678
443
5671
523
5272
309
4966
444
5911
295
4740
292
3359
243
4266
350
4644
226
3429


In [11]:
# Assuming your list of Galaxy objects is named 'gals_074'
for index, galaxy in enumerate(gals_074):
    print("Gal: [", index , ']')

Gal: [ 0 ]
Gal: [ 1 ]
Gal: [ 2 ]
Gal: [ 3 ]
Gal: [ 4 ]
Gal: [ 5 ]
Gal: [ 6 ]
Gal: [ 7 ]
Gal: [ 8 ]
Gal: [ 9 ]
Gal: [ 10 ]
Gal: [ 11 ]
Gal: [ 12 ]
Gal: [ 13 ]
Gal: [ 14 ]
Gal: [ 15 ]
Gal: [ 16 ]
Gal: [ 17 ]
Gal: [ 18 ]
Gal: [ 19 ]
Gal: [ 20 ]
Gal: [ 21 ]
Gal: [ 22 ]
Gal: [ 23 ]
Gal: [ 24 ]
Gal: [ 25 ]
Gal: [ 26 ]
Gal: [ 27 ]
Gal: [ 28 ]
Gal: [ 29 ]
Gal: [ 30 ]
Gal: [ 31 ]
Gal: [ 32 ]
Gal: [ 33 ]
Gal: [ 34 ]
Gal: [ 35 ]
Gal: [ 36 ]
Gal: [ 37 ]
Gal: [ 38 ]
Gal: [ 39 ]
Gal: [ 40 ]
Gal: [ 41 ]
Gal: [ 42 ]
Gal: [ 43 ]
Gal: [ 44 ]
Gal: [ 45 ]
Gal: [ 46 ]
Gal: [ 47 ]
Gal: [ 48 ]
Gal: [ 49 ]
Gal: [ 50 ]
Gal: [ 51 ]
Gal: [ 52 ]
Gal: [ 53 ]
Gal: [ 54 ]
Gal: [ 55 ]
Gal: [ 56 ]
Gal: [ 57 ]
Gal: [ 58 ]
Gal: [ 59 ]
Gal: [ 60 ]
Gal: [ 61 ]
Gal: [ 62 ]
Gal: [ 63 ]
Gal: [ 64 ]
Gal: [ 65 ]
Gal: [ 66 ]
Gal: [ 67 ]
Gal: [ 68 ]
Gal: [ 69 ]
Gal: [ 70 ]
Gal: [ 71 ]
Gal: [ 72 ]
Gal: [ 73 ]
Gal: [ 74 ]
Gal: [ 75 ]
Gal: [ 76 ]
Gal: [ 77 ]
Gal: [ 78 ]
Gal: [ 79 ]
Gal: [ 80 ]
Gal: [ 81 ]
Gal: [ 82 ]
Gal: [ 83 ]
Ga

In [12]:
# try different method to get specs:
#specs = np.vstack([g.stars.get_spectra_incident(grid).lnu for g in gals_074])
#print(specs)

In [18]:
g = gals_074[137]
print(g)

In [19]:
print(type(g))

<class 'synthesizer.particle.galaxy.Galaxy'>


In [20]:
print(g.stars.initial_masses)


[35784444. 10046882. 35729828. 24234642. 28452670. 26559054. 20050398.
 28736250. 28479760. 31694732. 13933997. 25009610. 23550900. 16070881.
 20925284. 21255214. 31317708. 20077630. 26360406. 20204594. 24747696.
 24181034. 19314928. 23426320. 21775454. 31532226. 25994324. 24603788.
 17130586. 24406598. 17439444. 26653368. 37304372. 30248602. 28654848.
 22211396. 26582392. 31003906. 19641748. 26947720. 24179976. 21050012.
 21305606. 32369742. 22641816. 34298952. 29898632. 15424778. 18634790.
 20170590. 25213044. 14839478. 25185608. 19328002. 25405636. 18164074.
 23087704. 31421402. 32947592. 30431308. 23710870. 27279966. 28912910.
 19775624. 29653470. 27943758. 15706061. 28598480. 26599314. 30384394.
 33439152. 26910966. 20623334. 35205424. 19659044. 24442396. 28492834.
 23413170. 32271832. 32252990.  7430425. 35945540. 28166552. 18873400.
 23230088. 32815802. 15253708. 28593720. 28720990. 36279220. 26684304.
 19127744. 30529028. 25074568. 16075368. 10969922. 27358878. 16372039.
 28395

In [21]:
print(g.stars.ages)


[5.66565548e+09 4.53630143e+09 6.41140474e+09 7.23948484e+09
 6.57153101e+09 6.74208004e+09 5.58427051e+09 1.14218202e+09
 7.60378655e+09 7.59819233e+09 1.13993208e+09 7.32474402e+09
 5.33428723e+09 5.32819442e+09 6.26101138e+09 2.46171815e+08
 7.59594772e+09 5.84981009e+08 6.19419381e+09 5.06321095e+09
 8.04879665e+09 1.43550109e+09 5.41408980e+09 6.81364025e+09
 8.22381764e+09 6.83517935e+09 6.84586583e+09 4.54915297e+09
 5.54858836e+09 1.49500583e+09 1.42676122e+09 6.59754230e+09
 2.33032595e+09 4.38123258e+09 2.56863193e+09 4.97251002e+09
 7.26511687e+09 4.37363335e+09 4.47151594e+09 5.62801099e+09
 4.33546255e+09 5.62520564e+09 4.44160648e+09 6.50004777e+09
 7.90712497e+08 1.24735044e+09 3.10995604e+08 5.66809645e+08
 2.64897078e+07 5.35137929e+08 1.32127485e+09 4.79502225e+08
 1.46167062e+09 4.84282184e+08 1.03137523e+09 4.55576365e+08
 1.57998897e+09 5.20896619e+09 5.08111336e+09 3.22657849e+09
 3.56606278e+09 2.52770881e+09 4.61108315e+09 5.39017929e+09
 3.60558253e+09 1.877663

In [22]:
print(g.stars.metallicities)

[3.18072666e-03 5.57395723e-03 3.62171466e-03 3.33565357e-03
 2.34390516e-03 3.39714182e-03 4.79144230e-03 2.15196540e-03
 1.63525797e-03 1.72037701e-03 2.63621379e-03 3.37845157e-03
 5.76980691e-03 5.33247460e-03 3.95644084e-03 3.90741788e-03
 1.53119559e-03 4.36312007e-03 5.51010389e-03 5.77896647e-03
 4.19687538e-04 1.61052449e-03 1.39559206e-05 2.59958756e-06
 7.98888067e-10 1.57155027e-05 9.44466970e-04 4.50819614e-04
 2.60537886e-03 1.38388551e-03 1.67325197e-03 1.90138852e-03
 1.40516052e-03 5.01146133e-04 1.43561745e-03 1.43631805e-05
 8.19841528e-10 1.20059948e-03 6.90769157e-05 2.72337911e-06
 2.20863032e-03 2.72636044e-06 3.58425576e-04 1.64353033e-03
 3.46188736e-03 2.25300761e-03 5.03063668e-03 4.10348177e-03
 4.22061794e-03 4.05838201e-03 4.29489557e-03 4.55625961e-03
 3.32188443e-03 4.20670956e-03 4.57713334e-03 5.25696576e-03
 3.27596976e-03 2.59908265e-03 2.39696749e-03 3.14964424e-03
 2.85351346e-03 3.66501277e-03 3.26114963e-03 4.19129909e-04
 2.08370993e-03 3.432296


for gal in gals_074:
    print(f"Galaxy has {gal.stars.nstars} stellar particles")
    print(f"Galaxy gas {gal.gas.nparticles} gas particles")


In [23]:


cat_074 = dir_+'/groups_074.hdf5'
# open file
f_h5py = h5py.File(cat_074, 'r')

# read different attributes of the header
boxSize_074 = f_h5py['Header'].attrs[u'BoxSize']/1e3 #Mpc/h
redshift_074 = f_h5py['Header'].attrs[u'Redshift']


# Get the number of star particles for each galaxy
nstars_list = [gal.stars.nstars for gal in gals_074]
# Get the number of gas particles for each galaxy
ngas_list = [gal.gas.nparticles for gal in gals_074]

# Calculate the minimum and maximum number of star particles
min_nstars = min(nstars_list)
max_nstars = max(nstars_list)

# Calculate the minimum and maximum number of gas particles
min_ngas = min(ngas_list)
max_ngas = max(ngas_list)

print(f"Minimum number of star particles: {min_nstars}")
print(f"Maximum number of star particles: {max_nstars}")
print(f"Minimum number of gas particles: {min_ngas}")
print(f"Maximum number of gas particles: {max_ngas}")


In [24]:

spec_list = []
for index, gal in enumerate(gals_074):
    print("Gal: [", index , ']')
    print('in for loop')
    # get_spectra_incident An Sed object containing the stellar spectra
    spec = gal.stars.get_spectra_incident(grid)
    print(spec)
    print('got spectra, getting fnu')
    spec.get_fnu0()
    print('appending spec to list')
    spec_list.append(spec)



Gal: [ 0 ]
in for loop
----------
SUMMARY OF SED 
Number of wavelength points: 401 
Wavelength range: [1400.00 Å,             1600.00 Å] 
log10(Peak luminosity/erg/(Hz*s)):             27.73 
log10(Bolometric luminosity/erg/s):42.0988898319883----------
got spectra, getting fnu
appending spec to list
Gal: [ 1 ]
in for loop
----------
SUMMARY OF SED 
Number of wavelength points: 401 
Wavelength range: [1400.00 Å,             1600.00 Å] 
log10(Peak luminosity/erg/(Hz*s)):             27.64 
log10(Bolometric luminosity/erg/s):42.01350139417053----------
got spectra, getting fnu
appending spec to list
Gal: [ 2 ]
in for loop
----------
SUMMARY OF SED 
Number of wavelength points: 401 
Wavelength range: [1400.00 Å,             1600.00 Å] 
log10(Peak luminosity/erg/(Hz*s)):             26.96 
log10(Bolometric luminosity/erg/s):41.303999283190855----------
got spectra, getting fnu
appending spec to list
Gal: [ 3 ]
in for loop
----------
SUMMARY OF SED 
Number of wavelength points: 401 
Wavelen

----------
SUMMARY OF SED 
Number of wavelength points: 401 
Wavelength range: [1400.00 Å,             1600.00 Å] 
log10(Peak luminosity/erg/(Hz*s)):             27.74 
log10(Bolometric luminosity/erg/s):42.11413761540538----------
got spectra, getting fnu
appending spec to list
Gal: [ 86 ]
in for loop
----------
SUMMARY OF SED 
Number of wavelength points: 401 
Wavelength range: [1400.00 Å,             1600.00 Å] 
log10(Peak luminosity/erg/(Hz*s)):             28.64 
log10(Bolometric luminosity/erg/s):42.98530591150269----------
got spectra, getting fnu
appending spec to list
Gal: [ 87 ]
in for loop
----------
SUMMARY OF SED 
Number of wavelength points: 401 
Wavelength range: [1400.00 Å,             1600.00 Å] 
log10(Peak luminosity/erg/(Hz*s)):             27.76 
log10(Bolometric luminosity/erg/s):42.11570270670913----------
got spectra, getting fnu
appending spec to list
Gal: [ 88 ]
in for loop
----------
SUMMARY OF SED 
Number of wavelength points: 401 
Wavelength range: [1400.00 

In [25]:
filt1 = Filter("top_hat/filter.1", lam_min=1400, lam_max=1600, new_lam=grid.lam)
filt_lst = [filt1]
# combine
seds = combine_list_of_seds(spec_list)

seds.lnu # rest frame lumd
seds.get_photo_luminosities(filt_lst)
seds.photo_luminosities.photo_luminosities
abs_mag = lnu_to_absolute_mag(seds.photo_luminosities.photo_luminosities)
abs_mag_th = abs_mag[0]


In [26]:
# next steps, get luminosity function for these magnitudes
# co-moving volume: BoxSize_025 and redshift:
little_h =  0.6711
Vphys = (boxSize_074/little_h )**3
Vcom = Vphys * ((1+redshift_074)**3)


In [27]:

massBinLimits = np.arange(-22, -16, 0.5)
phi, phi_sigma, hist = calc_lf(abs_mag_th, Vcom, massBinLimits)
# NOTE: 074 is the same redshift as CV_0/025
massBinLimits = massBinLimits[:-1]


In [28]:
hist

array([ 0.,  1.,  1.,  5.,  9., 12.,  8.,  7., 16., 15., 14.])

In [29]:
phi_sigma

array([0.00000000e+00, 1.22980409e-05, 1.22980409e-05, 2.74992554e-05,
       3.68941227e-05, 4.26016633e-05, 3.47841124e-05, 3.25375578e-05,
       4.91921636e-05, 4.76301076e-05, 4.60150555e-05])

In [ ]:
output_dir = "/home/jovyan/camels/LH/get_LF/output/"

# Define output file path
output_file = f"{output_dir}{LH_X}_test.txt"

# Write the data to the text file line by line
with open(output_file, 'w') as txtfile:
    # Write phi values
    txtfile.write("phi\n")
    for value in phi_l:
        txtfile.write(f"{value}\n")

    # Write phi_sigma values
    txtfile.write("phi_sigma\n")
    for value in phi_sigma_l:
        txtfile.write(f"{value}\n")

    # Write hist values
    txtfile.write("hist\n")
    for value in hist:
        txtfile.write(f"{value}\n")

    # Write massBinLimits values
    txtfile.write("massBinLimits\n")
    for value in massBinLimits:
        txtfile.write(f"{value}\n")


print('Written out: /home/jovyan/camels/play/synth-play/LH/output/',LH_X)

# Stop the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")

In [ ]:
# Get memory usage in bytes
memory_usage = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
print("Memory usage end:", memory_usage/1000000, "GB")


In [ ]:
# will do latest LH_ set in loop as a test
label_z = 'z = 0.46'


# Plot the luminosity function
plt.errorbar(massBinLimits, phi, yerr=phi_sigma, fmt='o', color='blue',label=label_z)
plt.xlabel('Absolute Magnitude (AB)')
plt.ylabel('Number Density (Mpc^-3 mag^-1)')
plt.yscale('log')

plt.title('Luminosity Function XMM-OM filter')
plt.grid(True)
plt.show()